In [10]:
from biocrnpyler import MultiMixtureGraph, Mixture, Component, Mechanism, Species, Compartment, Complex, Reaction

 # Diffusion on a Lattice

In [2]:
s = Species("substrate")

mixtures = []
compartment_names= []

mixtures.append(Mixture("m1"))
mixtures.append(Mixture("m2"))
mixtures.append(Mixture("m3"))
mixtures.append(Mixture("m4"))
mixtures.append(Mixture("m5"))
mixtures.append(Mixture("m6"))
mixtures.append(Mixture("m7"))
mixtures.append(Mixture("m8"))
mixtures.append(Mixture("m9"))

mmg = MultiMixtureGraph("mmg")

for mixture in mixtures:
    mixture.add_species(s)
    m, cn, c = mmg.add_mixture(mixture)
    compartment_names.append(cn)

mmg.connect(compartment_names[0], compartment_names[1], "diff1", "diff1")
mmg.connect(compartment_names[0], compartment_names[4], "diff2", "diff2")
mmg.connect(compartment_names[0], compartment_names[3], "diff3", "diff3")
mmg.connect(compartment_names[1], compartment_names[2], "diff4", "diff4")
mmg.connect(compartment_names[1], compartment_names[3], "diff5", "diff5")
mmg.connect(compartment_names[1], compartment_names[5], "diff6", "diff6")
mmg.connect(compartment_names[2], compartment_names[5], "diff7", "diff7")
mmg.connect(compartment_names[2], compartment_names[4], "diff8", "diff8")
mmg.connect(compartment_names[5], compartment_names[8], "diff9", "diff9")
mmg.connect(compartment_names[5], compartment_names[7], "diff10", "diff10")
mmg.connect(compartment_names[4], compartment_names[8], "diff11", "diff11")
mmg.connect(compartment_names[4], compartment_names[7], "diff12", "diff12")
mmg.connect(compartment_names[4], compartment_names[6], "diff13", "diff13")
mmg.connect(compartment_names[3], compartment_names[7], "diff14", "diff14")
mmg.connect(compartment_names[3], compartment_names[6], "diff15", "diff15")

CRN = mmg.compile_crn() 

print("CRN STRING REPRESENTATION: \n",CRN, "\n\n")
print("CRN PRETTY-PRINT REPRESENTATION:\n",
      CRN.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))


passive diffusion added
CRN STRING REPRESENTATION: 
 Species = substrate_m1_1, substrate_m2_1, substrate_m3_1, substrate_m4_1, substrate_m5_1, substrate_m6_1, substrate_m7_1, substrate_m8_1, substrate_m9_1
Reactions = [
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate
	substrate <--> substrate


# Membrane Transport

In [6]:
class MembraneTransport(Mechanism):
    def __init__(self, name, transport_protein, type = "transport_with_protein", **keywords):
        if isinstance(transport_protein, Species):
            self.transport_protein = transport_protein
        else:
            raise ValueError("'transport_protein' must be a Species!")
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)
        
    def update_species(self, int_species, ext_species):
        species = [int_species, ext_species, self.transport_protein]
        species += [Complex([int_species, self.transport_protein])]
        return species
        
    def update_reactions(self, int_species, ext_species, component, part_id = None):
        kb =(component.get_parameter("kb", part_id = part_id, mechanism  = self)).value
        ku = (component.get_parameter("ku", part_id = part_id, mechanism = self)).value
        kt = (component.get_parameter("kt", part_id = part_id, mechanism = self)).value
        
    
        comp = Complex([int_species, self.transport_protein])
        
        binding_rxn = Reaction.from_massaction(inputs=[int_species, self.transport_protein], outputs=[comp], k_forward=kb, k_reverse=ku)
        transport_rxn = Reaction.from_massaction(inputs=[comp], outputs=[self.transport_protein, ext_species], k_forward=kt)
    
        return [binding_rxn, transport_rxn]
    
    
class MembraneTransportProtein(Component):
    
    def __init__(self, name, int_m, ext_m, transport_prot, **keywords):
        self.internal = self.set_species(int_m, material_type = "molecule")
        self.external = self.set_species(ext_m, material_type = "molecule")
        self.transport_protien = self.set_species(transport_prot, material_type = "protien")
        Component.__init__(self = self, name = name, **keywords)
    
    def update_species(self):
        mech_transport = self.get_mechanism("transport_with_protein")
        return mech_transport.update_species(self.internal, self.external)
    
    def update_reactions(self):
        mech_transport = self.get_mechanism("transport_with_protein")
        return mech_transport.update_reactions(self.internal, self.external, component = self, part_id = self.name)

In [12]:
internal = Compartment("internal")
external = Compartment("external")
s_int = Species("substrate", compartment = internal)
s_ext = Species("substrate", compartment = external)

M_int = Mixture("internal_mx")
M_int.add_species(s_int)
M_ext = Mixture("external_mx")
M_ext.add_species(s_ext)

transport_protein = Species("transport_prot", material_type = "protein")

params = {"kb": 0.3, "ku": 0.7, "kt": 0.5}

mechanism = MembraneTransport("transport_with_protein", transport_protein) 
transport_molecules =  MembraneTransportProtein("membrane1", s_int, s_ext, transport_protein, parameters = params)
M_memb = Mixture("transport_mixture", components = [transport_molecules], parameters = params, mechanisms = mechanism)


mmg = MultiMixtureGraph("membrane_transport_graph")
m1, cn1, c1 = mmg.add_mixture(M_int, compartment = internal)
m2, cn2, c2 = mmg.add_mixture(M_ext, compartment = external)
m3, cn3, c3 = mmg.add_mixture(M_memb, compartment = "membrane")
mmg.connect(cn1, cn3, "external_membrane", "internal")
mmg.connect(cn2, cn3, "internal_membrane", "exernal")


CRN2 = mmg.compile_crn(add_passive_diffusion = False)

print("CRN STRING REPRESENTATION: \n", CRN2, "\n\n")
print("CRN PRETTY-PRINT REPRESENTATION:\n",
      CRN2.pretty_print(show_rates = True, show_attributes = True, show_materials = True, show_keys = False))


CRN STRING REPRESENTATION: 
 Species = substrate_internal, substrate_external, protein_transport_prot_membrane_1, complex_protein_transport_prot_membrane_1_substrate_internal_membrane_1_, complex_protein_transport_prot_membrane_1_substrate_internal_
Reactions = [
	substrate+protein[transport_prot] <--> complex[protein[transport_prot]:substrate]
	complex[protein[transport_prot]:substrate] --> protein[transport_prot]+substrate
] 


CRN PRETTY-PRINT REPRESENTATION:
 Species(N = 5) = {
protein[transport_prot] (@ 0),  substrate (@ 0),  substrate (@ 0),  complex[protein[transport_prot]:substrate] (@ 0),  complex[protein[transport_prot]:substrate] (@ 0),  
}

Reactions (2) = [
0. substrate+protein[transport_prot] <--> complex[protein[transport_prot]:substrate]
 Kf=k_forward * substrate_internal * protein_transport_prot_membrane_1
 Kr=k_reverse * complex_protein_transport_prot_membrane_1_substrate_internal_
  k_forward=0.3
  k_reverse=0.7

1. complex[protein[transport_prot]:substrate] --> prot